In [ ]:
from IPython.display import display
import fitz
from PIL import Image
import numpy as np
import os
import json


pdf_path = "/Users/vasyl/Desktop/LDL/LAFLA/files from Brianna & instructions/sample_imput_file.pdf"
# pdf_path = '/Users/vasyl/Desktop/LDL/LAFLA/files from Brianna & instructions/Dfs Proposed Jury Docs - LACIV244_David Wilson.pdf'
# pdf_path = '/Users/vasyl/Desktop/LDL/LAFLA/files from Brianna & instructions/proposed LACIV 244_Charles Baker.pdf'

# Pages 6, 7, 8, 9, 10, 11 in human numbering -> indices 5, 6, 7, 8, 9, 10
pages_1based = [6, 7, 8, 9, 10, 11]

# Crop parameters
crop_size = 20  # default 20x20
half_size = 10  # used for center-based crops

# Original right-edge x for page 6; we derive center x from this
x_right_default = 554

# Per-page y-values (human page numbers as keys).
# Page 6 values are the original top-left y for a 20px box; convert to centers by adding half_size
y_values_by_page = {
    6: [y + half_size for y in [
        1949, 2022, 2085, 2146, 2208, 2270, 2343, 2430, 2513, 2576, 2636, 2698, 2774
    ]],
    7: [
        960, 1035, 1112, 1188, 1354, 1446, 1524, 1587, 1660, 1738, 1812, 1903,
        1983, 2057, 2148, 2227, 2289, 2363, 2441, 2515, 2595, 2655 , 2719 , 2780 , 2842 , 2903
    ],
    8: [
        960, 1022, 1170, 1262, 1343, 1415, 1523, 1631, 1739, 1847, 1965,
        2114, 2265, 2356, 2463, 2562, 2635, 2726, 2834
    ],
    9: [
        970, 1063, 1217, 1307, 1474, 1565, 1657, 1748, 1840, 1931,
        2024, 2114, 2205, 2297, 2388, 2480, 2571, 2663, 2754, 2845
    ],
    10: [
        971, 1064, 1156, 1247, 1338, 1430
    ],
    11: [
        1692, 1787, 1984, 2090, 2199
    ],
}

# Page-specific labels
labels_by_page = {
    6: [
        100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112
    ],
    7: [
        113, 114, 115, 116, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209,
        210, 211, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222
    ],
    8: [
        223, 224, 4300, 4301, 4302, 4303, 4304, 4305, 4306, 4307, 4308,
        4309, 4320, 4321, 4322, 4323, 4324, 4325, 4326
    ],
    9: [
        4327, 4328, 4340, 4341, 5000, 5001, 5002, 5003, 5004, 5005,
        5006, 5007, 5008, 5009, 5010, 5011, 5012, 5013, 5014, 5015
    ],
    10: [
        5016, 5017, 5018, 5019, 5020, 5090
    ],
    11: [
        "5000-VF", "5001-VF", "4300-VF", "4301-VF", "4302-VF"
    ],
}

# Pages 6, 7, 8, 9, 10, and 11 use centered cropping (±10 px each side)
# For page 6, center x is derived from right-edge by subtracting half_size
x_center_by_page = {
    6: x_right_default - half_size,
    7: 564,
    8: 564,
    9: 564,
    10: 554,
    11: 566,
}

# Accumulate results for JSON export
results = []

with fitz.open(pdf_path) as doc:
    for page_num in pages_1based:
        idx = page_num - 1
        if idx < 0 or idx >= doc.page_count:
            continue

        # Print page header before processing
        print(f"Page ------- {page_num} ------------------------------")

        # Choose y-values and labels for this page
        y_values = y_values_by_page[page_num]
        labels = labels_by_page[page_num]

        page = doc.load_page(idx)
        pix = page.get_pixmap(dpi=300)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img_gray = img.convert('L')

        # Center-based crop (symmetric +/- half_size)
        x_c = x_center_by_page[page_num]
        for label, y_c in zip(labels, y_values):
            l = max(0, x_c - half_size)
            r = min(img_gray.width, x_c + half_size)
            t = max(0, y_c - half_size)
            b = min(img_gray.height, y_c + half_size)

            crop_img = img_gray.crop((l, t, r, b))
            arr = np.asarray(crop_img, dtype=np.float32) / 255.0
            inv = 1.0 - arr
            inv_img = Image.fromarray((inv * 255).astype(np.uint8))
            display(inv_img)

            s = float(inv.sum())
            score = int(round(s * s))
            checked = score > 500
            print(label, score, checked)
            results.append({"label": str(label), "value": score, "checked": bool(checked)})

# Save results to JSON
out_dir = os.path.dirname(pdf_path)
base = os.path.splitext(os.path.basename(pdf_path))[0]
out_path = os.path.join(out_dir, f"{base}_checkbox_results_p{pages_1based[0]}-{pages_1based[-1]}.json")
with open(out_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nSaved results to: {out_path}")
print("\nSummary (label, value, checked):")
for r in results:
    print(f"{r['label']}\t{r['value']}\t{r['checked']}")

Page ------- 6 ------------------------------


100 1316 True


101 1772 True


102 1772 True


103 0 False


...

5000-VF 0 False


5001-VF 0 False


4300-VF 3164 True


4301-VF 3260 True


4302-VF 0 False

Saved results to: /Users/vasyl/Desktop/LDL/LAFLA/files from Brianna & instructions/sample_imput_file_checkbox_results_p6-11.json

Summary (label, value, checked):
100	1316	True
101	1772	True
102	1772	True
103	0	False
104	1772	True
105	0	False
106	1772	True
107	1772	True
108	1772	True
109	0	False
110	0	False
111	1772	True
112	1772	True
113	3264	True
114	3368	True
115	0	False
116	3264	True
200	3264	True
201	0	False
202	2707	True
203	3264	True
204	3034	True
205	2261	True
206	0	False
207	0	False
208	0	False
209	3368	True
210	3264	True
211	0	False
212	3264	True
213	2938	True
215	0	False
216	0	False
217	0	False
218	0	False
219	0	False
220	0	False
221	0	False
222	0	False
223	3168	True
224	0	False
4300	2707	True
4301	0	False
4302	3168	True
4303	3034	True
4304	0	False
4305	0	False
4306	0	False
4307	0	False
4308	0	False
4309	0	False
4320	3368	True
4321	3264	True
4322	0	False
4323	3366	True
4324	0	False
4325	2936	True
4326	0	False
4327	3364	True
4328	0	False
4340	3164	True
4341	0